## Import package

In [10]:
import sys
sys.path.append('./script')
from pyomo_solving import *
from ET_optme import *
import pandas as pd
import cobra
import gurobipy
import json
import multiprocessing
import os
from multiprocessing import Pool
from sympy import subsets
import pandas as pd
import matplotlib.pyplot as plt
import re

## setting path for models and data, example: C.glutamicum --->L_Lyscine

In [11]:
model_file = "./file/lys.json"
model0_file = './file/iCW773_uniprot_modification_del.json'
reaction_kcat_MW_file = './file/reaction_change_by_enzuse_PDH_n.csv'
dictionarymodel_path = './file/dictionarymodel_iCW.json'
reaction_g0_file=os.path.join('./file/iCW773_uniprot_modification_del_reaction_g0.csv')
metabolites_lnC_file = os.path.join('./file/metabolites_lnC_cg1.txt')
path_results = './results'
savepath = './results/picture'

In [12]:
# model、substrate、product、oxygence
inputdic = {"model":'lys.json',
"substrate":"EX_glc_e_reverse",
"biomass": "CG_biomass_cgl_ATCC13032",
 "product": "EX_lys_L_e",
"taskname": "ET-FSEOF",
"mode":"SET", # here "SET" for enzyme-thermo algirithm
"oxygenstate":"aerobic"}
# save task.json
with open('./results/task_fseof.json', 'w') as json_file:
    json.dump(inputdic, json_file, indent=4)
path_strain = 'iCW'

## invoke ET-FSEOF and run the task 

## 1.enzyme-thermo Model Construction ( this step may takes 10 minutes)

In [13]:
# Read the model
model = cobra.io.load_json_model(model_file)

# Load the model data into Get_Concretemodel_Need_Data
Concretemodel_Need_Data = Get_Concretemodel_Need_Data(model_file)

model0 = cobra.io.load_json_model(model0_file)

In [14]:
# Store the protein-centered model as a dictionary
dictionary_model = json_load(dictionarymodel_path)
dictionary_model.keys()
rname3 = []

# Store the protein-centered dictionary into Get_Concretemodel_Need_Data
get_dictionarymodel_data2(dictionary_model, Concretemodel_Need_Data, rname3)

# Load thermodynamic data into Get_Concretemodel_Need_Data
Get_Concretemodel_Need_Data_g0(Concretemodel_Need_Data, reaction_g0_file, metabolites_lnC_file, reaction_kcat_MW_file)

Concretemodel_Need_Data['reaction_g0']['g0'] = Concretemodel_Need_Data['reaction_g0']['g0'].replace(0, np.nan)
Concretemodel_Need_Data['reaction_g0'].dropna(subset=['g0'], inplace=True)

Concretemodel_Need_Data['reaction_g0'].at['AIRC3_reverse', 'g0'] = 0
Concretemodel_Need_Data['reaction_g0'].at['ATPS4rpp_reverse_num2', 'g0'] = 0

Inc = Concretemodel_Need_Data['metabolites_lnC']
for i in Concretemodel_Need_Data['metabolite_list']:
    if i not in Inc.index:
        Inc.loc[i, 'lnClb'] = -14.508658
        Inc.loc[i, 'lnCub'] = -3.912023

Concretemodel_Need_Data['metabolites_lnC'] = Inc

# Modify the model's oxygen conditions
if inputdic['oxygenstate'] == 'aerobic':
    model.reactions.get_by_id('EX_o2_e_reverse').upper_bound = 1000
if inputdic['oxygenstate'] == 'micro_aerobic': 
    model.reactions.get_by_id('EX_o2_e_reverse').upper_bound = 2 
if inputdic['oxygenstate'] == 'anaerobic': 
    model.reactions.get_by_id('EX_o2_e_reverse').upper_bound = 0


## 2.Algorithm :  ET-FSEOF

In [15]:
# Calculate the maximum production rate
product,objvalue2 = calculate_product_fseof(Concretemodel_Need_Data,inputdic,model)

3.1133333333333315


In [16]:
# Calculating the growth rate under the maximum production rate.
FSEOFdf,reactiondf = biomass(product,inputdic,Concretemodel_Need_Data,objvalue2,model,model0)

0.42872386022139214
0.42872386022139214
0.42872386022143827
0.11259605704807821
0.41400955125699784
0.3912038284231641
0.3912038284231641
0.38962403761059317
0.10535609503306301
0.3760600205569381
0.3547557975334794
0.3547557975334794
0.3516413527886899
0.09985708366597423
0.3434148805490105
0.3172357657352062
0.3172357657352062
0.31254153017792735
0.0949810832925884
0.3114026273409869
0.27665485165508297
0.27665485165508297
0.269364540983065
0.08973295540182756
0.26715261817734987
0.2327200517581727
0.2327200517581727
0.22360199381606968
0.0845502264125159
0.21627360309982738
0.18882774080288128
0.18882774080288128
0.1778394466490744
0.08201595836278072
0.17155165951209456
0.14616885845096342
0.14616885845096342
0.13338440082970907
0.08015799056574881
0.12957878167516104
0.09051473234712093
0.09051473234712093
0.06970041409587022
0.0735359268189304
0.07149049460416114
0.029837613257577937
0.029837613257577937
0.006469763745071331
0.06673065157802252
0.010422417082023183


## Analyze the result and generate table

In [17]:
columns = FSEOFdf.columns
sorted_columns = sorted(columns, key=lambda x: get_sort_key(x, FSEOFdf))
sorted_FSEOFdf = FSEOFdf[sorted_columns]
FSEOFdf = sorted_FSEOFdf[['gene'] + [col for col in sorted_columns if col != 'gene']]
#  determine the manipulation strategies
FSEOFdf =detail(FSEOFdf)
#  get the mean flux
FSEOFdf =result(FSEOFdf)
# save the result table 
output_fseof(path_results,inputdic,reactiondf,FSEOFdf)

In [18]:
# show the key results 
FSEOFdf[['gene', 'manipulations']]

,gene,manipulations
0,Cgl0588,down
1,Cgl2844,down
2,Cgl2143,unchanged
3,Cgl2196,unchanged
4,Cgl1503,unchanged
...,...,...
639,cgtRNA_3566,unchanged
640,cgtRNA_3568,unchanged
641,cgtRNA_3569,unchanged
642,Cgl1629,unchanged
